In [1]:
# imports and settings

import os
import time
import pickle
import warnings
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from copy import deepcopy

import numpy as np
from numpy import linalg as LA
from numpy import histogram2d

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, butter, filtfilt, welch
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile
from scipy.stats import wasserstein_distance_nd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import utils as ut
%load_ext autoreload
%autoreload 2

# do not show warnings
warnings.filterwarnings("ignore")

print("Imports complete.")

Imports complete.
Settings: height=800, width=1400, font_size=16
Imports complete.


In [2]:
# parameters and processing loop
height = 800
width = 1400
font_size = 16
fft_nperseg = 16000
percent_overlap = 0.0
window = 'hamming'
remove_dc = 20
crop_freq = 4000  # Hz, None to disable
normalization_window_size = 9  # in samples, None to disable
detection_threshold = 2
p_gap = 0.2
sigma = 0.1  # band_width = normalization_window_size // 2
p_scale = 1
slice_len = 10  # seconds
nbins = 10
graph_threshold = 1 / (nbins**2)

In [3]:
# folder_path = '../data/croatia/2407_1_600m'
# folder_path = '../data/croatia/2407_2_snake_route'
# folder_path = '../data/croatia/2307_freediving'
# folder_path = '../data/scooter_12062025_experiment'
folder_path = '../data/croatia/2507_1_1000m'
file_list = os.listdir(folder_path)
file_list = [f for f in file_list if f.endswith('.wav')]
file_list.sort()
file_list = file_list  # [:-3]
print(file_list[:5])

['RBW6737_20250725_080100.wav', 'RBW6737_20250725_080200.wav', 'RBW6737_20250725_080300.wav', 'RBW6737_20250725_080400.wav', 'RBW6737_20250725_080500.wav']


In [4]:
test_file = file_list[28]
fs, data = wavfile.read(os.path.join(folder_path, test_file))

F, T, Sxx, phase = ut.calc_spectrogram(data, fs, nperseg=fft_nperseg, percent_overlap=percent_overlap, window=window, remove_dc=remove_dc, crop_freq=crop_freq)
pxx = ut.calc_welch_from_spectrogram(Sxx)
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], shared_yaxes=True, horizontal_spacing=0.02)
fig.add_trace(go.Heatmap(z=Sxx, x=T, y=F, colorscale='Viridis', showscale=False, colorbar=dict(title="Spectrogram")), row=1, col=1)
fig.add_trace(go.Scatter(y=F, x=pxx, orientation='h', marker=dict(color='blue'), name="PSD"), row=1, col=2)
fig.update_layout(height=600, width=1000, title_text="Spectrogram and PSD", showlegend=False)
fig.update_xaxes(title_text="Time [s]", row=1, col=1)
fig.update_yaxes(title_text="Frequency [Hz]", row=1, col=1)
fig.show()

In [5]:
f0_test_list = [272, 312, 408, 592, 632, 640, 848, 856, 1080, 1264, 1280, 1704, 2560]  # Hz
test_nbins = 10

fig = make_subplots(rows = 2, cols=len(f0_test_list))
for i, f0 in enumerate(f0_test_list, start=1):
    f_idx = np.argmin(np.abs(F - f0))
    x = phase[f_idx, :]

    # calc transition matrix
    x = ut.normalize_data(x)
    x = ut.quantize_data(x, test_nbins)
    x_transitions = ut.get_s2g_transition_matrix(x, test_nbins)
    x_transitions = x_transitions / np.sum(x_transitions)

    # calc original K
    edge_count = np.count_nonzero(x_transitions > graph_threshold)
    K = edge_count / x_transitions.size

    # calc wasserstein distance from uniform distribution
    uniform_dist = np.ones_like(x_transitions) / x_transitions.size
    W = wasserstein_distance_nd(x_transitions.flatten(), uniform_dist.flatten())

    fig.add_trace(go.Heatmap(z=x_transitions, colorscale='Viridis', showscale=False), row=2, col=i)
    fig.update_xaxes(title_text=f"f={f0}, K={K:.4f}, W={W:.4f}", row=2, col=i)

    nodes = np.unique(x)
    edges = ut.get_s2g_edges(x)

    # build graph from edge list instead of from_numpy_array (which expects an adjacency matrix)
    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)

    # draw original graph (expects a plotly Figure)
    fig_g = ut.draw_graph(G)
    for tr in fig_g.data:
        fig.add_trace(deepcopy(tr), row=1, col=i)
    # use f0 as key into k (dict keyed by frequencies) to avoid KeyError
    fig.update_xaxes(title_text=f"f={f0}", row=1, col=i)

fig.update_layout(height=800, width=300*len(f0_test_list), title_text=f"Phase Signal Analysis for file={test_file}", showlegend=False)
fig.show()

In [6]:
def calc_all_K(data, fs):
    F, T, Sxx, phase = ut.calc_spectrogram(data, fs, nperseg=fft_nperseg, percent_overlap=percent_overlap, window=window, remove_dc=remove_dc, crop_freq=crop_freq)
    Ks = []
    for f0 in F:
        f_idx = np.argmin(np.abs(F - f0))
        x = phase[f_idx, :]
        x_transitions = ut.get_s2g(x, n_levels=nbins)
        k = ut.get_K(x_transitions)
        Ks.append(k)
    return Ks

In [7]:
K = np.zeros((len(file_list), len(F)))
for i, file in enumerate(file_list):
    fs, data = wavfile.read(os.path.join(folder_path, file))
    ks = calc_all_K(data, fs)
    K[i, :] = ks

In [8]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=K.T, x=np.arange(len(file_list)), y=F, colorscale='Viridis', showscale=True, colorbar=dict(title="K value")))
fig.update_layout(title="K values over time and frequency", xaxis_title="File index", yaxis_title="Frequency [Hz]", height=600, width=1000)
fig.show()